# CNN MODEL


In [ ]:
!pip install tensorflow opencv-python roboflow scikit-learn matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


## install dependencies

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

## preparing classification

In [ ]:
# === Define Class Names ===
class_names = ['glass', 'leaf', 'metal', 'paper', 'plastic']
num_classes = len(class_names)

waste_category = {
    "glass": ("non-biodegradable", "recyclable"),
    "leaf": ("biodegradable", "non-recyclable"),   # typically organic waste
    "metal": ("non-biodegradable", "recyclable"),
    "paper": ("biodegradable", "recyclable"),
    "plastic": ("non-biodegradable", "recyclable"),
}

## data prep

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="kllPn6XsBsLz8K2iQx6i")
project = rf.workspace("deep-learning-rp9gw").project("final-proj-jpjtg")
version = project.version(3)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to final-proj-3 in yolov8:: 100%|██████████| 25292/25292 [00:06<00:00, 3711.55it/s]


## image set up

In [ ]:
image_dir = '/content/final-proj-3/train/images'
label_dir = '/content/final-proj-3/train/labels'
image_size = (90, 90)

## loading images and annotations

In [ ]:
X = []
y_class = []
y_bbox = []

for filename in os.listdir(image_dir):
    if filename.endswith(('.jpg', '.png')):
        img_path = os.path.join(image_dir, filename)
        label_path = os.path.join(label_dir, os.path.splitext(filename)[0] + ".txt")

        # Load image
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, image_size)
        img = img / 255.0  # Normalize

        # Check if label file exists
        if not os.path.exists(label_path):
            continue

        with open(label_path, 'r') as f:
            lines = f.readlines()
        if not lines:
            continue

        # Use only the first object in the label file
        line = lines[0].strip().split()
        if len(line) == 5:
            cls_id = int(line[0])  # class id from label file
            bbox = list(map(float, line[1:]))

            X.append(img)
            y_class.append(cls_id)
            y_bbox.append(bbox)

X = np.array(X)
y_class = to_categorical(y_class, num_classes=num_classes)
y_bbox = np.array(y_bbox)

## data split

In [ ]:
X_temp, X_test, y_cls_temp, y_cls_test, y_bbox_temp, y_bbox_test = train_test_split(
    X, y_class, y_bbox, test_size=0.1, random_state=42, stratify=y_class
)

X_train, X_val, y_cls_train, y_cls_val, y_bbox_train, y_bbox_val = train_test_split(
    X_temp, y_cls_temp, y_bbox_temp, test_size=0.1111, random_state=42
)

In [ ]:
from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import Callback

class TrainingLogger(Callback):
    def __init__(self, X_val, y_val, class_names, log_file='training_log.txt'):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.class_names = class_names
        self.log_file = log_file

        # Initialize log file
        with open(self.log_file, 'w') as f:
            f.write("Epoch\tVal_Loss\tVal_Accuracy\tVal_F1_Score\n")

    def on_epoch_end(self, epoch, logs=None):
        y_true = np.argmax(self.y_val, axis=1)
        y_pred_probs = self.model.predict(self.X_val, verbose=0)[0]  # [0] for classification output
        y_pred = np.argmax(y_pred_probs, axis=1)

        f1 = f1_score(y_true, y_pred, average='weighted')
        acc = logs.get('val_class_output_accuracy', 0)
        loss = logs.get('val_loss', 0)

        print(f"\n📘 Epoch {epoch + 1} Validation Log:")
        print(f"   - val_loss: {loss:.4f}")
        print(f"   - val_accuracy: {acc:.4f}")
        print(f"   - val_f1_score: {f1:.4f}\n")

        with open(self.log_file, 'a') as f:
            f.write(f"{epoch + 1}\t{loss:.4f}\t{acc:.4f}\t{f1:.4f}\n")

logger = TrainingLogger(X_val, y_cls_val, class_names)

## CNN Model Architecture

In [ ]:
# Define IMG_SIZE and NUM_CLASSES before they are used
IMG_SIZE = image_size[0]  # Or image_size[1], assuming image_size is (height, width)
NUM_CLASSES = num_classes # Use the already defined num_classes

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

model.summary()

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 88, 88, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 88, 88, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 44, 44, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 42, 42, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 42, 42, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 21, 21, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 19, 19, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 19, 19, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 9, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 10368)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,327,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,422,021 (5.42 MB)

 Trainable params: 1,421,573 (5.42 MB)

 Non-trainable params: 448 (1.75 KB)

## Compile Model

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Callbacks

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.2)

## Train Model

In [ ]:
history = model.fit(X_train, y_cls_train,
                    validation_data=(X_val, y_cls_val),
                    epochs=50,
                    batch_size=8, # Changed from 32
                    callbacks=[early_stop, reduce_lr])

Epoch 1/50
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 268s 228ms/step - accuracy: 0.5060 - loss: 2.2725 - val_accuracy: 0.6024 - val_loss: 0.9127 - learning_rate: 0.0010
Epoch 2/50
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 266s 232ms/step - accuracy: 0.5910 - loss: 1.0247 - val_accuracy: 0.6190 - val_loss: 1.0775 - learning_rate: 0.0010
Epoch 3/50
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 323s 232ms/step - accuracy: 0.6139 - loss: 0.9704 - val_accuracy: 0.6321 - val_loss: 1.0239 - learning_rate: 0.0010
Epoch 4/50
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 268s 234ms/step - accuracy: 0.6257 - loss: 0.9113 - val_accuracy: 0.6617 - val_loss: 0.7752 - learning_rate: 0.0010
Epoch 5/50
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 324s 236ms/step - accuracy: 0.6357 - loss: 0.8768 - val_accuracy: 0.6486 - val_loss: 0.8092 - learning_rate: 0.0010
Epoch 6/50
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 319s 233ms/step - accuracy: 0.6286 - loss: 0.9088 - val_accuracy: 0.5693 - val_loss: 1.1936 - learning_rate: 0.0010
Epoch 7/50
1147/1147 ━━━━━━━━━━━━━━━━━━━━ 323s 235ms

## Evaluate Model

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score

# Get model predictions (probabilities)
y_val_pred_prob = model.predict(X_val)

# Convert predicted probabilities to class indices
y_val_pred = np.argmax(y_val_pred_prob, axis=1)

# Convert true labels to class indices if they are one-hot encoded
if len(y_cls_val.shape) > 1 and y_cls_val.shape[1] > 1:
    y_val_true = np.argmax(y_cls_val, axis=1)
else:
    y_val_true = y_cls_val

# Now y_val_true and y_val_pred are defined — run classification metrics

print(classification_report(y_val_true, y_val_pred))

accuracy = accuracy_score(y_val_true, y_val_pred)
print(f"Accuracy: {accuracy:.4f}")

precision_macro = precision_score(y_val_true, y_val_pred, average='macro')
recall_macro = recall_score(y_val_true, y_val_pred, average='macro')
f1_macro = f1_score(y_val_true, y_val_pred, average='macro')

precision_micro = precision_score(y_val_true, y_val_pred, average='micro')
recall_micro = recall_score(y_val_true, y_val_pred, average='micro')
f1_micro = f1_score(y_val_true, y_val_pred, average='micro')

precision_weighted = precision_score(y_val_true, y_val_pred, average='weighted')
recall_weighted = recall_score(y_val_true, y_val_pred, average='weighted')
f1_weighted = f1_score(y_val_true, y_val_pred, average='weighted')

print(f"Precision (macro): {precision_macro:.4f}")
print(f"Recall (macro): {recall_macro:.4f}")
print(f"F1 Score (macro): {f1_macro:.4f}")

print(f"Precision (micro): {precision_micro:.4f}")
print(f"Recall (micro): {recall_micro:.4f}")
print(f"F1 Score (micro): {f1_micro:.4f}")

print(f"Precision (weighted): {precision_weighted:.4f}")
print(f"Recall (weighted): {recall_weighted:.4f}")
print(f"F1 Score (weighted): {f1_weighted:.4f}")


## Plot Accuracy & Loss

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')
plt.show()

In [ ]:
# === Evaluate on test set ===
X_test = []
y_cls_test = []
y_bbox_test = []

for i in range(len(test_gen)):
    X_batch, y_batch = test_gen[i]
    X_test.append(X_batch)
    y_cls_test.append(y_batch['class_output'])
    y_bbox_test.append(y_batch['bbox_output'])

X_test = np.vstack(X_test)
y_cls_test = np.vstack(y_cls_test)
y_bbox_test = np.vstack(y_bbox_test)

pred_class_probs, pred_bbox = model.predict(X_test)
y_true = np.argmax(y_cls_test, axis=1)
y_pred = np.argmax(pred_class_probs, axis=1)

print("Classification Report:\n", classification_report(y_true, y_pred, target_names=class_names))
print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
print(f"F1 Score (weighted): {f1_score(y_true, y_pred, average='weighted'):.4f}")